### 연관성 분석 측정지표

상품 추천, 즉, 고객이 함께 많이 구매할 것 같은 상품을 예측하는 데에 가장 많이 활용되는 척도로는 지지도(Support), 신뢰도(Confidence), 향상도(Lift)라는 것이 있다.


- 지지도: 사과와 우유를 얼마나 함께 구매하는지를 나타내는 지표라고 볼 수 있다. 지지도는 좋은 규칙(구성비가 높고 빈도가 많은)을 찾거나 불필요한 연산을 줄이기 위한 기준으로 사용된다. A와 B를 모두 포함하는 거래의 수/전체 거래의 수


- 신뢰도: 사과를 많이 구매한 사람 중 얼마나 많은 사람이 우유도 함께 구매했을까? 확률을 구해보니 사과를 구매한 사람은 3명이고, 그 중 2명이 우유를 구매했다. 그럼 여기서 구할 수 있는 신뢰도는 66.7%이다. 지지도와 달리 confidence(A->B)의 값과 confidence(B->A)의 값이 다르고 값이 1에 가까울수록 연관성이 높다.


- 향상도: 지지도가 별로 팔리지 않는 상품을 솎아내기 위한 지표였다면 거꾸로 많이 팔리는 인기상품을 골라낼 때 활용되는 것이 향상도다. A를 구매하지 않았을 때 품목 B를 구매할 확률 대비, A를 구매했을 때 품목 B의 구매 확률의 증가 비율을 의미한다.(A와 B를 모두 포함하는 거래의 수 * 전체 거래의 수)/(A를 포함하는 거래의 수*B를 포함하는 거래의 수)

    향상도가 1이면 두 품목은 독립이고, 1보다 작다면 음의 상관관계로, A를 구매하면 B를 구매하지 않을 확률이 구매할 확률보다 큼을 의미한다. 향상도가 1보다 크면 두 품목은 양의 상관관계로, 임의로 B를 구매할 확률보다 A를 구매한 후 B를 구매할 확률이 큼을 의미한다.

In [1]:
!pip install mlxtend

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [34]:
dataset = [['사과','치즈','생수'],
           ['생수','호두','치즈','고등어'],
           ['수박','사과','생수'],
           ['생수','호두','치즈','옥수수']]

In [35]:
te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary, columns = te.columns_)
df

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [36]:
# 지지도 0.5 이상인 것들만 출력
# 전체 거래에서 item이 같이 거래될 확률?
# apriori : ~에 앞서는.. 상품들 중 많이 팔리는 것?
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(사과, 생수)"
5,0.75,"(생수, 치즈)"
6,0.50,"(생수, 호두)"
7,0.50,"(호두, 치즈)"
8,0.50,"(생수, 호두, 치즈)"


In [11]:
# 신뢰도가 0.3 이상인 것들만 출력
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
1,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
2,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0
3,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf
4,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
5,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
6,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
7,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
8,"(생수, 호두)",(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
9,"(생수, 치즈)",(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5


Q. 아래 dataset으로 연관성 분석을 수행하세요

In [12]:
dataset = [['Milk','Onion','Nutmeg','Eggs','Yogurt'],
           ['Onion','Nutmeg','Eggs','Yogurt'],
           ['Milk','Apple','Eggs'],
           ['Milk','Unicorn','Corn','Yogurt'],
           ['Corn','Onion','Onion','Ice cream','Eggs']]

In [14]:
te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Eggs,Ice cream,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,True,False,True,True,True,False,True
1,False,False,True,False,False,True,True,False,True
2,True,False,True,False,True,False,False,False,False
3,False,True,False,False,True,False,False,True,True
4,False,True,True,True,False,False,True,False,False


In [16]:
# 지지도 0.5 이상인 것들만 출력
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,0.6,(Milk)
2,0.6,(Onion)
3,0.6,(Yogurt)
4,0.6,"(Onion, Eggs)"


In [21]:
#                                 기준이 되는 column, 기준column의 최소값
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
1,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6


[과제]
6개 이상의 연관규칙을 구하세요.

In [22]:
dataset = [['아메리카노','카페라떼'],
          ['카페라떼','아메리카노','카푸치노'],
          ['바닐라라떼','아메리카노'],
          ['아메리카노','카페라떼'],
          ['카페라떼','아메리카노','카푸치노'],
          ['바닐라라떼','아메리카노'],
          ['녹차라떼','카페라떼','아메리카노'],
          ['카페모카','아메리카노'],
          ['아메리카노','카페라떼'],
          ['초콜릿','아메리카노'],
          ['아메리카노'],
          ['카페모카','카페라떼']]

In [23]:
te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,녹차라떼,바닐라라떼,아메리카노,초콜릿,카페라떼,카페모카,카푸치노
0,False,False,True,False,True,False,False
1,False,False,True,False,True,False,True
2,False,True,True,False,False,False,False
3,False,False,True,False,True,False,False
4,False,False,True,False,True,False,True
5,False,True,True,False,False,False,False
6,True,False,True,False,True,False,False
7,False,False,True,False,False,True,False
8,False,False,True,False,True,False,False
9,False,False,True,True,False,False,False


In [30]:
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.166667,(바닐라라떼)
1,0.916667,(아메리카노)
2,0.583333,(카페라떼)
3,0.166667,(카페모카)
4,0.166667,(카푸치노)
5,0.166667,"(바닐라라떼, 아메리카노)"
6,0.500000,"(카페라떼, 아메리카노)"
7,0.166667,"(카푸치노, 아메리카노)"
8,0.166667,"(카푸치노, 카페라떼)"
9,0.166667,"(카푸치노, 카페라떼, 아메리카노)"


In [32]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(바닐라라떼),(아메리카노),0.166667,0.916667,0.166667,1.0,1.090909,0.013889,inf
1,(카푸치노),(아메리카노),0.166667,0.916667,0.166667,1.0,1.090909,0.013889,inf
2,(카푸치노),(카페라떼),0.166667,0.583333,0.166667,1.0,1.714286,0.069444,inf
3,"(카푸치노, 카페라떼)",(아메리카노),0.166667,0.916667,0.166667,1.0,1.090909,0.013889,inf
4,"(카푸치노, 아메리카노)",(카페라떼),0.166667,0.583333,0.166667,1.0,1.714286,0.069444,inf
5,(카푸치노),"(카페라떼, 아메리카노)",0.166667,0.500000,0.166667,1.0,2.000000,0.083333,inf
